In [1]:
from datetime import datetime

import numpy as np
import comet_ml
import torch
from scipy import stats
from sklearn.model_selection import *
from skorch import NeuralNetClassifier, callbacks

from code_S3R import my_nets
import code_S3R.my_utils.other_utils as utils

In [2]:
hyper_params = [
    {
        'max_epochs': [5], 'batch_size': [32],
        'lr': [0.0001],
        'module__preprocess': [None],
        'module__conv_type': ['regular'],
        'module__channel_list': [
            [22],
        ],
        'module__activation_fct': ['prelu'],
        'module__dropout': [0.4],
    },
]

# -------------
# Data
# -------------

# Load the dataset
x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = utils.load_data()
# Shuffle sequences and resize sequences
x_train, x_test, y_train_14, y_train_28, y_test_14, y_test_28 = utils.preprocess_data(x_train, x_test,
                                                                                      y_train_14,
                                                                                      y_train_28,
                                                                                      y_test_14, y_test_28,
                                                                                      temporal_duration=100)

# Feeding it PyTorch tensors doesn't seem to work, but numpy arrays with the right format is okay
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train_14 = y_train_14.astype(np.long)
y_test_14 = y_test_14.astype(np.long)
y_train_28 = y_train_28.astype(np.long)
y_test_28 = y_test_28.astype(np.long)

y_train = y_train_14
y_test = y_test_14


/Users/alexandre/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [3]:
# -------------
# Perform grid search
# -------------

device = 'cuda' if torch.cuda.is_available() else 'cpu'
search_run_id = '{:%m%d_%H%M%S}'.format(datetime.now())  # unique identifier for the grid_search / random_search run

net = NeuralNetClassifier(
    module=my_nets.Net,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('my_cb', utils.MyCallback(params_to_log=utils.get_param_keys(hyper_params), search_run_id=search_run_id,
                                   log_to_comet_ml=False, log_to_csv=True)),
        ('early_stopping', callbacks.EarlyStopping(patience=50))
    ],
    device=device
)
net.set_params(callbacks__print_log=None)  # deactivate default score printing each epoch

gs = GridSearchCV(estimator=net, param_grid=hyper_params, refit=False, scoring='accuracy',
                  verbose=2, cv=5, error_score=0)

# gs = RandomizedSearchCV(estimator=net, param_distributions=hyper_params, refit=False, scoring='accuracy',
#                        verbose=2, cv=5, error_score=0)

gs.fit(x_train, y_train)

# -------------
# Save and log results
# -------------

utils.save_and_print_results(search_run_id=search_run_id, cv_results=gs.cv_results_, grid_search_params=hyper_params)


TypeError: __init__() got an unexpected keyword argument 'params_to_log'

In [7]:
net.history

In [13]:
import pandas as pd
data = [13, 4, 2, 5, 2]*100
data2 = [13, 4, 2, 5, 2]*100

In [14]:
df = pd.DataFrame({
    'params': {
        'lr': 0.3,
        'net_type': 'bla'
    },
    'data': {
        'data': data,
        'data2': data2
    }
})

In [25]:
df.to_csv()

',data,params\ndata,"[13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13, 4, 2, 5, 2, 13